In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 19.8 MB/s 


In [3]:
# !pip install --pre torch torchvision -f https://download.pytorch.org/whl/nightly/cu111/torch_nightly.html -U

In [4]:
######################################################################################################
#  ________   _________   ___    ___ ___  ___  ________  ________  ___  __      _______   _______
# |\   ___  \|\___   ___\|\  \  /  /|\  \|\  \|\   __  \|\   ____\|\  \|\  \   /  ___  \ /  ___  \
# \ \  \\ \  \|___ \  \_|\ \  \/  / | \  \\\  \ \  \|\  \ \  \___|\ \  \/  /|_/__/|_/  //__/|_/  /|
#  \ \  \\ \  \   \ \  \  \ \    / / \ \   __  \ \   __  \ \  \    \ \   ___  \__|//  / /__|//  / /
#   \ \  \\ \  \   \ \  \  /     \/   \ \  \ \  \ \  \ \  \ \  \____\ \  \\ \  \  /  /_/__  /  /_/__
#    \ \__\\ \__\   \ \__\/  /\   \    \ \__\ \__\ \__\ \__\ \_______\ \__\\ \__\|\________\\________\
#     \|__| \|__|    \|__/__/ /\ __\    \|__|\|__|\|__|\|__|\|_______|\|__| \|__| \|_______|\|_______|
#                        |__|/ \|__|
######################################################################################################

import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import random
from torch.utils.data import Dataset, DataLoader
from mne.decoding import Vectorizer
torch.manual_seed(42)

In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


In [6]:
# Path to training data
# train_path = "/content/gdrive/My Drive/codalab/training/"
train_path = "/home/deepak/learning_project/student/BrainAge/training/"

In [7]:
# Path to testing data (public test set)
# test_path = "/content/gdrive/My Drive/codalab/testing_flat/"
# test_path = "/home/deepak/learning_project/student/BrainAge/testing_flat/"

In [8]:
# Path to final testing data (public test set)
# final_test_path = "/content/gdrive/My Drive/codalab/validation-final/"
# final_test_path = "/home/deepak/learning_project/student/BrainAge/validation-final/"

In [9]:
condition_ec = "EC"  # closed eyes condition
condition_eo = "EO"  # closed eyes condition
train_subj = 1100  # use 1100 of the 1200 training subjects for training
val_subj = 100   # use 100 of the 1200 training subjects for validation
test_subj = 400  # use 10 instead of 400 testing subjects, for demonstration purpose

In [10]:
class CustomDataset(Dataset):
    def __init__(self, path, subj, ages, start):
        self.path = path
        self.subj = subj
        self.ages = ages
        self.start = start
    def __len__(self):
        return self.subj
    def __getitem__(self, idx):
        s = idx + self.start
        fname = f"subj{s:04}_{condition_eo}_raw.fif.gz"
        raw = mne.io.read_raw(self.path + fname, preload=True, verbose='warning')
        d = raw.get_data()
        ft = d.shape[-1]
        data_eo = torch.zeros(1, 129, 10000)
        data_eo[:, :, :ft] = -200 * torch.tensor(d)
        fname = f"subj{s:04}_{condition_ec}_raw.fif.gz"
        raw = mne.io.read_raw(self.path + fname, preload=True, verbose='warning')
        d = raw.get_data()
        ft = d.shape[-1]
        data_ec = torch.zeros(1, 129, 20000)
        data_ec[:, :, :ft] = -200 * torch.tensor(d)
        data = (data_eo, data_ec)
        age = self.ages[idx]
        return data, age

In [11]:
# get the age to predict from the CSV file
meta = pd.read_csv(train_path + "train_subjects.csv")
y_train = []
for age in meta.age[:1200]:
    y_train.append(age)
print(np.min(y_train), np.mean(y_train), np.median(y_train), np.max(y_train))

5.005932 10.356727775833333 9.587952999999999 21.899041


In [12]:
train_data = CustomDataset(train_path, train_subj, y_train[:train_subj], 1)

In [13]:
val_data = CustomDataset(train_path, val_subj, y_train[train_subj:train_subj+val_subj], 1 + train_subj)

In [14]:
# DataLoader
batch_size = 8

train_loader = DataLoader(
    train_data,
    batch_size=batch_size,
    num_workers=1
)

val_loader = DataLoader(
    val_data,
    batch_size=batch_size,
    num_workers=1
)

In [15]:
# final_test_ages = [0 for i in range(test_subj)]
# final_test_data = CustomDataset(final_test_path, test_subj, final_test_ages, 1201)

In [16]:
# test_loader = DataLoader(
#     final_test_data,
#     batch_size=batch_size,
#     num_workers=1
# )

In [17]:
class AgeNET(nn.Module):
    def conv_block(self, in_channels, out_channels, kernel, stride, pool):
        return nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel, stride=stride, padding=0),
            nn.LeakyReLU(),
            nn.AvgPool2d(kernel_size=(1, pool), stride=(1, pool))
        ).to(DEVICE)
    def __init__(self, sampling_rate, num_T, num_C):
        super().__init__()
        self.device = torch.device(DEVICE)
        print(DEVICE)
        self.to(DEVICE)
        self.pool = 4
        self.state = 0
        self.Time1_eo = self.conv_block(1, num_T, (1, sampling_rate//2), 1, self.pool*4)
        self.Time2_eo = self.conv_block(1, num_T, (1, sampling_rate//4), 1, self.pool*4)
        self.Time3_eo = self.conv_block(1, num_T, (1, sampling_rate//8), 1, self.pool*4)
        self.Time1_ec = self.conv_block(1, num_T, (1, sampling_rate//2), 1, self.pool*4)
        self.Time2_ec = self.conv_block(1, num_T, (1, sampling_rate//4), 1, self.pool*4)
        self.Time3_ec = self.conv_block(1, num_T, (1, sampling_rate//8), 1, self.pool*4)
        self.BN_T_eo = nn.BatchNorm2d(num_T).to(DEVICE)
        self.BN_T_ec = nn.BatchNorm2d(num_T).to(DEVICE)
        self.Chan1_eo = self.conv_block(num_T, num_C, (129, 1), 1, self.pool)
        self.Chan2_eo = self.conv_block(num_T, num_C, (65, 1), (64, 1), self.pool)
        # self.Chan3_eo = self.conv_block(num_T, num_C, (32, 1), (32, 1), self.pool)
        self.Chan1_ec = self.conv_block(num_T, num_C, (129, 1), 1, self.pool)
        self.Chan2_ec = self.conv_block(num_T, num_C, (65, 1), (64, 1), self.pool)
        # self.Chan3_ec = self.conv_block(num_T, num_C, (32, 1), (32, 1), self.pool)
        self.BN_C_eo = nn.BatchNorm2d(num_C).to(DEVICE)
        self.BN_C_ec = nn.BatchNorm2d(num_C).to(DEVICE)
        size_eo, size_ec = self.get_size()
        print(size_eo, size_ec)
        self.fc_eo = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=size_eo, out_features=1024, bias=True),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=False)
        )
        self.fc_ec = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=size_ec, out_features=2048, bias=True),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=False)
        )
        self.classifier = nn.Sequential(
            nn.Linear(in_features=3072, out_features=1024, bias=True),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=False),
            # nn.Linear(in_features=4096, out_features=2048, bias=True),
            # nn.BatchNorm1d(2048),
            # nn.ReLU(),
            # nn.Dropout(p=0.003, inplace=False),
            # nn.Linear(in_features=2048, out_features=1024, bias=True),
            # nn.BatchNorm1d(1024),
            # nn.ReLU(),
            # nn.Dropout(p=0.002, inplace=False),
            nn.Linear(in_features=1024, out_features=512, bias=True),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=512, out_features=256, bias=True),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(p=0.3, inplace=False),
            nn.Linear(in_features=256, out_features=64, bias=True),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )
#         self.last_layer = []
#         for j in out_len:
#             self.last_layer.append(
#                 nn.Sequential(
#                     nn.Linear(in_features=64, out_features=j, bias=True, device=DEVICE),
#                     nn.ReLU()
#                 )
#             )
        self.l_l0 = nn.Sequential(
                    nn.Linear(in_features=64, out_features=25, bias=True),
                    nn.ReLU()
                )
        self.l_l1 = nn.Sequential(
                    nn.Linear(in_features=64, out_features=1, bias=True),
                    nn.ReLU()
                )
    def forward(self, x_eo, x_ec):
        y = self.Time1_eo(x_eo)
        out_eo = y
        y = self.Time2_eo(x_eo)
        out_eo = torch.cat((out_eo, y), dim=-1)
        y = self.Time3_eo(x_eo)
        out_eo = torch.cat((out_eo, y), dim=-1)
        y = self.Time1_ec(x_ec)
        out_ec = y
        y = self.Time2_ec(x_ec)
        out_ec = torch.cat((out_ec, y), dim=-1)
        y = self.Time3_ec(x_ec)
        out_ec = torch.cat((out_ec, y), dim=-1)
        out_eo = self.BN_T_eo(out_eo)
        out_ec = self.BN_T_ec(out_ec)
        z = self.Chan1_eo(out_eo)
        out_f_eo = z
        z = self.Chan2_eo(out_eo)
        out_f_eo = torch.cat((out_f_eo, z), dim=2)
        # z = self.Chan3_eo(out_eo)
        # out_f_eo = torch.cat((out_f_eo, z), dim=2)
        z = self.Chan1_ec(out_ec)
        out_f_ec = z
        z = self.Chan2_ec(out_ec)
        out_f_ec = torch.cat((out_f_ec, z), dim=2)
        # z = self.Chan3_ec(out_ec)
        # out_f_ec = torch.cat((out_f_ec, z), dim=2)
        out_f_eo = self.BN_C_eo(out_f_eo)
        out_f_ec = self.BN_C_ec(out_f_ec)
        out = torch.cat((self.fc_eo(out_f_eo), self.fc_ec(out_f_ec)), dim=-1)
        out = self.classifier(out)
        if self.state == 0:
            out = self.l_l0(out)
        else:
            out = self.l_l1(out)
        return out
    def switch_state(self):
        self.state = 1 - self.state
    def set_state(self, st):
        self.state = st
    def get_state(self):
        return self.state
    def get_size(self):
        d_eo = torch.ones(1, 1, 129, 10000).to(DEVICE)
        d_ec = torch.ones(1, 1, 129, 20000).to(DEVICE)
        y = self.Time1_eo(d_eo)
        out_eo = y
        y = self.Time2_eo(d_eo)
        out_eo = torch.cat((out_eo, y), dim=-1)
        y = self.Time3_eo(d_eo)
        out_eo = torch.cat((out_eo, y), dim=-1)
        y = self.Time1_ec(d_ec)
        out_ec = y
        y = self.Time2_ec(d_ec)
        out_ec = torch.cat((out_ec, y), dim=-1)
        y = self.Time3_ec(d_ec)
        out_ec = torch.cat((out_ec, y), dim=-1)
        out_eo = self.BN_T_eo(out_eo)
        out_ec = self.BN_T_ec(out_ec)
        z = self.Chan1_eo(out_eo)
        out_f_eo = z
        z = self.Chan2_eo(out_eo)
        out_f_eo = torch.cat((out_f_eo, z), dim=2)
        # z = self.Chan3_eo(out_eo)
        # out_f_eo = torch.cat((out_f_eo, z), dim=2)
        z = self.Chan1_ec(out_ec)
        out_f_ec = z
        z = self.Chan2_ec(out_ec)
        out_f_ec = torch.cat((out_f_ec, z), dim=2)
        # z = self.Chan3_ec(out_ec)
        # out_f_ec = torch.cat((out_f_ec, z), dim=2)
        return torch.numel(out_f_eo), torch.numel(out_f_ec)

In [18]:
model_A = AgeNET(512, 9, 6).to(DEVICE)

cuda
8298 16740


In [19]:
model_A = model_A.to(DEVICE)

In [20]:
from torchsummary import summary

summary(model_A, [(1, 129, 10000), (1, 129, 20000)])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 9, 129, 9745]           2,313
         LeakyReLU-2         [-1, 9, 129, 9745]               0
         AvgPool2d-3          [-1, 9, 129, 609]               0
            Conv2d-4         [-1, 9, 129, 9873]           1,161
         LeakyReLU-5         [-1, 9, 129, 9873]               0
         AvgPool2d-6          [-1, 9, 129, 617]               0
            Conv2d-7         [-1, 9, 129, 9937]             585
         LeakyReLU-8         [-1, 9, 129, 9937]               0
         AvgPool2d-9          [-1, 9, 129, 621]               0
           Conv2d-10        [-1, 9, 129, 19745]           2,313
        LeakyReLU-11        [-1, 9, 129, 19745]               0
        AvgPool2d-12         [-1, 9, 129, 1234]               0
           Conv2d-13        [-1, 9, 129, 19873]           1,161
        LeakyReLU-14        [-1, 9, 129

In [21]:
def train(model, device, train_loader, val_loader, lr, epochs):
    train_abs_log = []
    train_log = []
    val_log = []
    val_abs_log = []
    abs_loss = nn.L1Loss(reduction='mean')
    lossfunc = nn.CrossEntropyLoss(reduction='mean')
    optimizer = torch.optim.NAdam(model.parameters(), lr=lr)
    f = plt.figure()
    xr = 0
    for epoch in range(epochs):
        train_abs_loss = 0.0
        train_loss = 0.0
        print(f"Epoch #{1 + epoch:02}: ")
        i = 0
        model.train()
        for data, age in train_loader:
            data_eo, data_ec = data
            data_eo = data_eo.to(device)           # shape = (batch_size, 1, 129, 10000)
            data_ec = data_ec.to(device)           # shape = (batch_size, 1, 129, 20000)
            batch_size = age.size(0)
            optimizer.zero_grad()
            output = model(data_eo, data_ec)       # shape = (batch_size, 25)
            if model.get_state() == 1:
                loss = abs_loss(torch.squeeze(output), age.float().to(device))
                train_abs_loss += loss.item() * batch_size
            else:
                loss = lossfunc(output, np.around(age).to(torch.int64).to(device))
                train_abs_loss += abs_loss(torch.argmax(output, dim=-1), age.to(DEVICE)).item() * batch_size
            loss.backward()
            optimizer.step()
            print(i)
            i += batch_size
            train_loss += loss.item() * batch_size
        train_abs_loss /= train_subj
        train_loss /= train_subj
        train_abs_log.append(train_abs_loss)
        train_log.append(train_loss)
        val_abs_loss = 0.0
        val_loss=0.0
        model.eval()
        for data, age in val_loader:
            data_eo, data_ec = data
            data_eo = data_eo.to(device)           # shape = (batch_size, 129, 10000)
            data_ec = data_ec.to(device)           # shape = (batch_size, 129, 20000)
            batch_size = age.size(0)
            output = model(data_eo, data_ec)       # shape = (batch_size, 25)
            if model.get_state() == 1:
                loss = abs_loss(torch.squeeze(output), age.float().to(device))
                val_abs_loss += loss.item() * batch_size
            else:
                loss = lossfunc(output, np.around(age).to(torch.int64).to(device))
                val_abs_loss += abs_loss(torch.argmax(output, dim=-1), age.to(DEVICE)).item() * batch_size

            val_loss += loss.item() * batch_size
        val_abs_loss /= val_subj
        val_loss /= val_subj
        val_abs_log.append(val_abs_loss)
        val_log.append(val_loss)
        print(f"CrossEntropyLoss:     train = {train_loss}, validation = {val_loss}")
        print(f"mean absolute error:  train = {train_abs_loss}, validation = {val_abs_loss}")
        plt.clf()
        plt.plot(train_log, label='train loss')
        plt.plot(val_log, label='validation loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.title('Loss')
        plt.legend()
        plt.show()
        xr += 1
        if xr % 5 == 0:
            model.switch_state()
    return train_abs_log, val_abs_log

In [ ]:
train_abs_log, val_abs_log = train(model_A, DEVICE, train_loader, val_loader, 0.004, 30)

Epoch #01: 
0
8


In [ ]:
def test(model, device, test_loader):
    y_pred = []
    model.eval()
    model.set_state(1)
    for data, age in test_loader:
        data_eo, data_ec = data
        data_eo = data_eo.to(device)           # shape = (batch_size, 129, 10000)
        data_ec = data_ec.to(device)           # shape = (batch_size, 129, 20000)
        x = torch.cat((data_eo, data_ec), -1)  # shape = (batch_size, 129, 30000)
        output = torch.squeeze(model(x))
        #print(output.shape)
        y_pred.extend(list(output.detach().cpu().numpy()))
    return y_pred

In [ ]:
torch.save(model_A.state_dict(), "/home/deepak/learning_project/model_scripted.pt")

In [ ]:
# y_pred = test(model_A, DEVICE, test_loader)

In [ ]:
# # create submission file
# submission = []
# for subj, pred in zip(range(1201, 1201 + test_subj), y_pred):
#     submission.append({"id": subj, "age": pred})
# pd.DataFrame(submission).to_csv("mysubmission3_0.csv", index=False)
#
# # zip the csv file (without anything else) and submit it on the website!